# 디노이징

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


## NAFnet 이미지 넣기

### Git clone

In [ ]:
!git clone https://github.com/megvii-research/NAFNet
%cd NAFNet

Cloning into 'NAFNet'...
remote: Enumerating objects: 517, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 517 (delta 157), reused 125 (delta 125), pack-reused 334
Receiving objects: 100% (517/517), 16.19 MiB | 24.38 MiB/s, done.
Resolving deltas: 100% (273/273), done.
/content/NAFNet


### 환경 설정

In [ ]:
!pip install -r requirements.txt
!pip install --upgrade --no-cache-dir gdown
!python3 setup.py develop --no_cuda_ext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 25.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ******************************

### pretrained 모델

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=14Fht1QQJ2gMlk4N1ERCRuElg8JfjrWWR', "./experiments/pretrained_models/", quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=14Fht1QQJ2gMlk4N1ERCRuElg8JfjrWWR
From (redirected): https://drive.google.com/uc?id=14Fht1QQJ2gMlk4N1ERCRuElg8JfjrWWR&confirm=t&uuid=f5a65cad-dd22-4879-ab10-ff630fe61f34
To: /content/NAFNet/experiments/pretrained_models/NAFNet-SIDD-width64.pth
100%|██████████| 464M/464M [00:05<00:00, 79.6MB/s]


'./experiments/pretrained_models/NAFNet-SIDD-width64.pth'

### 준비

In [ ]:
import torch

from basicsr.models import create_model
from basicsr.utils import img2tensor as _img2tensor, tensor2img, imwrite
from basicsr.utils.options import parse
import numpy as np
import cv2
import matplotlib.pyplot as plt

def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img
def img2tensor(img, bgr2rgb=False, float32=True):
    img = img.astype(np.float32) / 255.
    return _img2tensor(img, bgr2rgb=bgr2rgb, float32=float32)

def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1) 
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('NAFNet output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)

def single_image_inference(model, img, save_path):
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      sr_img = tensor2img([visuals['result']])
      imwrite(sr_img, save_path)


### 기본 모델

In [ ]:
opt_path = 'options/test/SIDD/NAFNet-width64.yml'
opt = parse(opt_path, is_train=False)
opt['dist'] = False
NAFNet = create_model(opt)

 load net keys <built-in method keys of collections.OrderedDict object at 0x7f13390bc6c0>


### inference

In [ ]:
import glob
import os
import numpy as np
from PIL import Image

image_path = '/content/drive/Othercomputers/내 노트북/dataset/sheep/lr'

img_list = os.listdir(image_path) #디렉토리 내 모든 파일 불러오기
img_list_png = [img for img in img_list if img.endswith(".png")] #지정된 확장자만 필터링

for i in img_list:
    input_path = ('/content/drive/Othercomputers/내 노트북/dataset/sheep/lr/'+i)

    print(input_path)
    img_input = imread(input_path)
    inp = img2tensor(img_input)
    single_image_inference(NAFNet, inp, '/content/drive/MyDrive/denoising/NAFNet/sheep/'+(i))

/content/drive/Othercomputers/내 노트북/dataset/sheep/lr/sheep (999).png
/content/drive/Othercomputers/내 노트북/dataset/sheep/lr/sheep (996).png
/content/drive/Othercomputers/내 노트북/dataset/sheep/lr/sheep (992).png
/content/drive/Othercomputers/내 노트북/dataset/sheep/lr/sheep (1004).png
/content/drive/Othercomputers/내 노트북/dataset/sheep/lr/sheep (998).png
/content/drive/Othercomputers/내 노트북/dataset/sheep/lr/sheep (991).png
/content/drive/Othercomputers/내 노트북/dataset/sheep/lr/sheep (984).png
/content/drive/Othercomputers/내 노트북/dataset/sheep/lr/sheep (990).png
/content/drive/Othercomputers/내 노트북/dataset/sheep/lr/sheep (983).png
/content/drive/Othercomputers/내 노트북/dataset/sheep/lr/sheep (986).png
/content/drive/Othercomputers/내 노트북/dataset/sheep/lr/sheep (1001).png
/content/drive/Othercomputers/내 노트북/dataset/sheep/lr/sheep (987).png
/content/drive/Othercomputers/내 노트북/dataset/sheep/lr/sheep (989).png
/content/drive/Othercomputers/내 노ᄐ

### YOLOv5 이미지 넣기

In [ ]:
!pip install ultralytics

In [ ]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5x')  # or yolov5n - yolov5x6, custom


for i in range(1, 1001):
  img=('/content/drive/MyDrive/denoising/NAFNet/sheep/sheep (%d).png'%(i))
  results=model(img)
  results.save(save_dir='/content/drive/MyDrive/denoising-/NAFNet/sheep->yolov5xdata', exist_ok=True)
  

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-5-2 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 


requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 
Saved 1 image to /content/drive/MyDrive/denoising-/NAFNet/sheep->yolov5xdata
Saved 1 image to /content/drive/MyDrive/denoising-/NAFNet/sheep->yolov5xdata
Saved 1 image to /content/drive/MyDrive/denoising-/NAFNet/sheep->yolov5xdata
Saved 1 image to /content/drive/MyDrive/denoising-/NAFNet/sheep->yolov5xdata
Saved 1 image to /content/drive/MyDrive/denoising-/NAFNet/sheep->yolov5xdata
Saved 1 image to /content/drive/MyDrive/denoising-/NAFNet/sheep->yolov5xdata
Saved 1 image to /content/drive/MyDrive/denoising-/NAFNet/sheep->yolov5xdata
Saved 1 image to /content/drive/MyDrive/denoising-/NAFNet/sheep->yolov5xdata
Saved 1 image to /content/drive/MyDrive/denoising-/NAFNet/sheep->yolov5xdata
Saved 1 image to /content/drive/MyDrive/denoising-/NAFNet/sheep->yolov5xdata
Saved 1 image to /content/drive/MyDrive/denoising-/NAFNet/sheep->yolov5xdata
Saved 1 image to /content/drive/MyDrive/denoising-/NAFNet/sheep->yolov